In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
# Load your dataset (assumed to be in a CSV file)
dataset_path = 'output_dataset_vocal.csv'
df = pd.read_csv(dataset_path)

# Extract features (gain values for each frequency range) and target variables
X = df[['unmastered_20-40', 'unmastered_40-80', 'unmastered_80-160','unmastered_160-300','unmastered_300-600','unmastered_600-1200','unmastered_1200-2400','unmastered_2400-5000','unmastered_5000-10000','unmastered_10000-20000']].values
y = df[['mastered_20-40', 'mastered_40-80', 'mastered_80-160','mastered_160-300','mastered_300-600','mastered_600-1200','mastered_1200-2400','mastered_2400-5000','mastered_5000-10000','mastered_10000-20000']].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Define the neural network architecture
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(10,)),  # Input layer with 10 features
    keras.layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons
    keras.layers.Dense(10)  # Output layer with 10 neurons (one for each frequency range)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Print the model summary
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 3114 (12.16 KB)
Trainable params: 3114 (12.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/50


2/2 [==============================] - 1s 211ms/step - loss: 469.3148 - mae: 18.2789 - val_loss: 417.6597 - val_mae: 16.9553
Epoch 2/50
2/2 [==============================] - 0s 46ms/step - loss: 428.8662 - mae: 17.3875 - val_loss: 382.5669 - val_mae: 16.1589
Epoch 3/50
2/2 [==============================] - 0s 32ms/step - loss: 391.5903 - mae: 16.5370 - val_loss: 350.5309 - val_mae: 15.3976
Epoch 4/50
2/2 [==============================] - 0s 48ms/step - loss: 358.2766 - mae: 15.7372 - val_loss: 320.8528 - val_mae: 14.6580
Epoch 5/50
2/2 [==============================] - 0s 40ms/step - loss: 327.1382 - mae: 14.9588 - val_loss: 292.3555 - val_mae: 13.9183
Epoch 6/50
2/2 [==============================] - 0s 32ms/step - loss: 298.5247 - mae: 14.2132 - val_loss: 266.2133 - val_mae: 13.2467
Epoch 7/50
2/2 [==============================] - 0s 40ms/step - loss: 272.8561 - mae: 13.5220 - val_loss: 242.2523 - val_mae: 12.6176
Epoch 8/50
2/2 [==============================] - 0s

In [9]:
# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer=ModelCheckpoint(filepath='Predictor_models/audio_predictor-vocal.hdf5',verbose=1,save_best_only=True)
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1, callbacks=checkpointer)

Epoch 1/500
1/2 [==============>...............] - ETA: 0s - loss: 0.3401 - mae: 0.4413
Epoch 1: val_loss improved from inf to 7.62267, saving model to Predictor_models\audio_predictor-vocal.hdf5
2/2 [==============================] - 0s 86ms/step - loss: 0.3084 - mae: 0.4165 - val_loss: 7.6227 - val_mae: 1.8394
Epoch 2/500
1/2 [==============>...............] - ETA: 0s - loss: 0.3182 - mae: 0.3882
Epoch 2: val_loss improved from 7.62267 to 7.61126, saving model to Predictor_models\audio_predictor-vocal.hdf5
2/2 [==============================] - 0s 45ms/step - loss: 0.3017 - mae: 0.3817 - val_loss: 7.6113 - val_mae: 1.8386
Epoch 3/500
1/2 [==============>...............] - ETA: 0s - loss: 0.2849 - mae: 0.3635
Epoch 3: val_loss did not improve from 7.61126
2/2 [==============================] - 0s 27ms/step - loss: 0.3155 - mae: 0.3975 - val_loss: 7.6993 - val_mae: 1.8220
Epoch 4/500
1/2 [==============>...............] - ETA: 0s - loss: 0.3208 - mae: 0.4037
Epoch 4: val_loss did not i

In [7]:
# Evaluate the model on the test data
from sklearn.metrics import r2_score
loss, mae = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)

# Calculate R-squared (Coefficient of Determination)
r_squared = r2_score(y_test, y_pred)
print(f"Test Loss: {loss}")
print(f"Test Mean Absolute Error: {mae}")
print(f"r2_score:{r_squared}")

1/1 [==============================] - 0s 19ms/step
Test Loss: 5.480091094970703
Test Mean Absolute Error: 1.6299608945846558
r2_score:0.9080314798842091
